In [1]:
import datetime
import time
import xlrd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


'''# 字符类型的时间
tss1 = '2010/04/09 04:44:11'
'''
#以x射线数据为准

#converters强制每个标签的数值类型
sheet = pd.read_excel('flare_euv.xlsx',converters={'DATE':str,'START':str,'PEAKT':str,'ENDT':str,'CLASS':str,'SIGN':int,'CLASS_N':float,'SUB':float,'STARTF':float,'ENDF':float,'CON':float,'T_D':int,'SIGN_2':int,'RATE':float})
print("sheet导入完成")
sheet1 = pd.read_excel('flare_x_new.xlsx',converters={'DATE':str,'START':str,'PEAKT':str,'ENDT':str,'SIGN':int})
print("sheet1导入完成")
##导入未筛选的Lya数据
sheet2 = pd.read_excel('euv_list_2010.xlsx',converters={'DATE':str,'TIME':str,'VALUE':float})
print("sheet2导入完成")
#导入未筛选的x数据
sheet2_x=pd.read_excel('goes_list_2010.xlsx',converters={'time_tag':str,'A_AVG':float,'B_AVG':float})
print("sheet2_x导入完成")
#sheet3 = pd.read_excel('euv_peak.xlsx')
l_index=range(120)
#print(l_index)
sheet3 = pd.DataFrame(columns=l_index)
sheet3_x = pd.DataFrame(columns=l_index)

#print(type(sheet3))
#print(type(sheet['PEAKT'][0]))
#sheet1.head()

i=0
j=0

while(sheet['N'][i]!=0):
    

    euv_start=sheet['DATE'][i].split(' ')[0]+' '+sheet['START'][i]  #将日期和时间整理拼接成字符串
    euv_peakt=sheet['DATE'][i].split(' ')[0]+' '+sheet['PEAKT'][i]
    euv_endt=sheet['DATE'][i].split(' ')[0]+' '+sheet['ENDT'][i]
    #print(type(sheet1['DATE'][j]))
    #print(j)
    #print(i)
    x_peakt=sheet1['DATE'][j].split(' ')[0]+' '+sheet1['PEAKT'][j]
#print(euv_start)
    timeArray1 = time.strptime(euv_start, "%Y-%m-%d %H:%M:%S")
    timeStamp1 = int(time.mktime(timeArray1))

    timeArray2 = time.strptime(euv_peakt, "%Y-%m-%d %H:%M:%S")
    timeStamp2 = int(time.mktime(timeArray2))

    timeArray3 = time.strptime(euv_endt, "%Y-%m-%d %H:%M:%S")
    timeStamp3 = int(time.mktime(timeArray3))


    timeArray4 = time.strptime(x_peakt, "%d-%m-%Y %H:%M:%S")  #将日期转化为TAI，秒数格式，唯一识别
    timeStamp4 = int(time.mktime(timeArray4))

    if(timeStamp4>timeStamp1) and (timeStamp4<timeStamp3): #比较x的峰值时间是否在ly耀斑的起始和结束时间段内
        #sheet['CLASS'][i]=sheet1['CLASS'][j]
        sheet1.loc[j,('SIGN')]=1
      
        i=i+1
        j=j+1
    elif timeStamp4<timeStamp1:
        j=j+1
    else:
        i=i+1

        
        
k=0
while(sheet1['N'][k]!=0):
    if(sheet1['CLASS'][k][0]=='M') or (sheet1['CLASS'][k][0]=='X') or (sheet1['CLASS'][k][0]=='C'):
        sheet1.loc[k,('SIGN')]=0  #将M和X级耀斑做标记，后期去掉
        k=k+1
        #print(k)
    elif sheet1['CLASS'][k][0]=='B' and sheet1['SIGN'][k]==1:#把GOES和Ly耀斑对应的B级耀斑筛选出来标记
        sheet1.loc[k,('SIGN')]=1  #将B级耀斑标记为1,后期画图分类
        k=k+1
    else:
        k=k+1

sheet1_new=sheet1
sheet1_new=sheet1_new[sheet1_new['SIGN'] == 1]     
        

#print(len(sheet1_new))        
sheet1_new.head(50)



#sheet2文件，存储每日的具体euv值

l=0
m=0
q=0
t=0
a=0
while(sheet2['N'][l]!=0):
    
    #print(m)
    euv_time=sheet2['DATE'][l].split(' ')[0]+' '+sheet2['TIME'][l]  #将日期和时间整理拼接成字符串
    euv_peakt_2=sheet1['DATE'][m].split(' ')[0]+' '+sheet1['PEAKT'][m]  #将日期和时间整理拼接成字符串
    #和x的峰值做比较
    #print(euv_start)
    timeArray5 = time.strptime(euv_time, "%Y-%m-%d %H:%M:%S")
    timeStamp5 = int(time.mktime(timeArray5))
    #timeStamp5=timeStamp5-30
    timeArray6 = time.strptime(euv_peakt_2,"%d-%m-%Y %H:%M:%S")
    timeStamp6 = int(time.mktime(timeArray6))
    #print(timeArray5)
    #print(timeArray6)
    if sheet1['SIGN'][m]==1: #在上面筛选出来的B级耀斑中研究
        #sheet['CLASS'][i]=sheet1['CLASS'][j]
        if (timeArray6==timeArray5):
            print('time:',int(sheet2['TIME'][l][3:5]))
            a=int(sheet2['TIME'][l][3:5]) #把对应时间的分钟数提取出来，减掉，以前面的小时数作为横轴数据起始点
            n=l-a
            for q in range(120): 
                if (sheet2['VALUE'][n]==-999):
                    sheet3.loc[t,q]=np.nan
                else:
                    sheet3.loc[t,q]=sheet2['VALUE'][n] #这里数据差30s，这里暂时补齐，最后一起还原
                n=n+1
            t=t+1
                
            l=l+1
            m=m+1
        else:
            l=l+1
        
    else:
        m=m+1

        

print('sheet3的长度：')        
print(len(sheet3))






#sheet2_x文件，存储每日的具体euv值

l_1=0
m_1=0
q_1=0
t_1=0
a_1=0
while(sheet2_x['B_NUM_PTS'][l_1]!=0):
    
    #print(m)
    x_time=sheet2_x['time_tag'][l_1]  #将日期和时间整理拼接成字符串
    x_peakt_2=sheet1['DATE'][m_1].split(' ')[0]+' '+sheet1['PEAKT'][m_1]  #将日期和时间整理拼接成字符串
    #和x的峰值做比较
    #print(euv_start)
    timeArray5 = time.strptime(x_time, "%Y-%m-%d %H:%M:%S")
    timeStamp5 = int(time.mktime(timeArray5))
    #timeStamp5=timeStamp5-30
    timeArray6 = time.strptime(x_peakt_2,"%d-%m-%Y %H:%M:%S")
    timeStamp6 = int(time.mktime(timeArray6))
    #print(timeArray5)
    #print(timeArray6)
    if sheet1['SIGN'][m_1]==1: #在上面筛选出来的B级耀斑中研究
        #sheet['CLASS'][i]=sheet1['CLASS'][j]
        if (timeArray6==timeArray5):
            print('x_time:',int(sheet2_x['time_tag'][l_1][14:16]))
            a_1=int(sheet2_x['time_tag'][l_1][14:16]) #把对应时间的分钟数提取出来，减掉，以前面的小时数作为横轴数据起始点
            n_1=l_1-a_1
            for q_1 in range(120): 
                if (sheet2_x['B_AVG'][n_1]==-99999):
                    sheet3_x.loc[t_1,q_1]=np.nan
                else:
                    sheet3_x.loc[t_1,q_1]=sheet2_x['B_AVG'][n_1] #这里数据差30s，这里暂时补齐，最后一起还原
                n_1=n_1+1
            t_1=t_1+1
                
            l_1=l_1+1
            m_1=m_1+1
        else:
            l_1=l_1+1
        
    else:
        m_1=m_1+1

print(sheet3)    
sheet3_x


sheet导入完成
sheet1导入完成
sheet2导入完成
sheet2_x导入完成
time: 55
time: 46
time: 29
time: 24
time: 38
time: 16
time: 39
time: 2
sheet3的长度：
8
x_time: 55
x_time: 46
x_time: 29
x_time: 24
x_time: 38
x_time: 16
x_time: 39
x_time: 2


KeyError: 396000

In [1]:
time=range(120)
y=sheet3_x.loc[0]
print(y)
plt.plot(time,y)

NameError: name 'sheet3_x' is not defined

In [ ]:
import datetime
import time
import xlrd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

sheet2_1 = pd.read_excel('euv_list_2011.xlsx',converters={'DATE':str,'TIME':str,'VALUE':float})
print("sheet2_1导入完成")
#导入未筛选的x数据
sheet2_1_x=pd.read_excel('goes_list_2011.xlsx',converters={'time_tag':str,'A_AVG':float,'B_AVG':float})
print("sheet2_1_x导入完成")
print('当前m值：')
pirnt(m)
#sheet2文件，存储每日的具体euv值

l=0

q=0

a=0
while(sheet2_1['N'][l]!=0):
    
    #print(m)
    euv_time=sheet2_1['DATE'][l].split(' ')[0]+' '+sheet2_1['TIME'][l]  #将日期和时间整理拼接成字符串
    euv_peakt_2=sheet1['DATE'][m].split(' ')[0]+' '+sheet1['PEAKT'][m]  #将日期和时间整理拼接成字符串
    #和x的峰值做比较
    #print(euv_start)
    timeArray5 = time.strptime(euv_time, "%Y-%m-%d %H:%M:%S")
    timeStamp5 = int(time.mktime(timeArray5))
    #timeStamp5=timeStamp5-30
    timeArray6 = time.strptime(euv_peakt_2,"%d-%m-%Y %H:%M:%S")
    timeStamp6 = int(time.mktime(timeArray6))
    #print(timeArray5)
    #print(timeArray6)
    if sheet1['SIGN'][m]==1: #在上面筛选出来的B级耀斑中研究
        #sheet['CLASS'][i]=sheet1['CLASS'][j]
        if (timeArray6==timeArray5):
            print('time:',int(sheet2_1['TIME'][l][3:5]))
            a=int(sheet2_1['TIME'][l][3:5]) #把对应时间的分钟数提取出来，减掉，以前面的小时数作为横轴数据起始点
            n=l-a
            for q in range(120): 
                if (sheet2_1['VALUE'][n]==-999):
                    sheet3.loc[t,q]=np.nan
                else:
                    sheet3.loc[t,q]=sheet2_1['VALUE'][n] #这里数据差30s，这里暂时补齐，最后一起还原
                n=n+1
            t=t+1
                
            l=l+1
            m=m+1
        else:
            l=l+1
        
    else:
        m=m+1

print('sheet3的长度：')        
print(len(sheet3))






#sheet2_1_x文件，存储每日的具体euv值

l_1=0
print('当前m_1值：',m_1)
q_1=0

a_1=0
while(sheet2_1_x['B_NUM_PTS'][l_1]!=0):
    
    #print(m)
    x_time=sheet2_1_x['time_tag'][l_1]  #将日期和时间整理拼接成字符串
    x_peakt_2=sheet1['DATE'][m_1].split(' ')[0]+' '+sheet1['PEAKT'][m_1]  #将日期和时间整理拼接成字符串
    #和x的峰值做比较
    #print(euv_start)
    timeArray5 = time.strptime(x_time, "%Y-%m-%d %H:%M:%S")
    timeStamp5 = int(time.mktime(timeArray5))
    #timeStamp5=timeStamp5-30
    timeArray6 = time.strptime(x_peakt_2,"%d-%m-%Y %H:%M:%S")
    timeStamp6 = int(time.mktime(timeArray6))
    #print(timeArray5)
    #print(timeArray6)
    if sheet1['SIGN'][m_1]==1: #在上面筛选出来的B级耀斑中研究
        #sheet['CLASS'][i]=sheet1['CLASS'][j]
        if (timeArray6==timeArray5):
            print('x_time:',int(sheet2_1_x['time_tag'][l_1][14:16]))
            a_1=int(sheet2_1_x['time_tag'][l_1][14:16]) #把对应时间的分钟数提取出来，减掉，以前面的小时数作为横轴数据起始点
            n_1=l_1-a_1
            for q_1 in range(120): 
                if (sheet2_1_x['B_AVG'][n_1]==-99999):
                    sheet3_x.loc[t_1,q_1]=np.nan
                else:
                    sheet3_x.loc[t_1,q_1]=sheet2_1_x['B_AVG'][n_1] #这里数据差30s，这里暂时补齐，最后一起还原
                n_1=n_1+1
            t_1=t_1+1
                
            l_1=l_1+1
            m_1=m_1+1
        else:
            l_1=l_1+1
        
    else:
        m_1=m_1+1

print(sheet3)    
sheet3_x


In [ ]:
import datetime
import time
import xlrd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

sheet2_2 = pd.read_excel('euv_list_2012.xlsx',converters={'DATE':str,'TIME':str,'VALUE':float})
print("sheet2_2导入完成")
#导入未筛选的x数据
sheet2_2_x=pd.read_excel('goes_list_2012.xlsx',converters={'time_tag':str,'A_AVG':float,'B_AVG':float})
print("sheet2_2_x导入完成")
print('当前m值：',m)

#sheet2文件，存储每日的具体euv值

l=0

q=0

a=0
while(sheet2_2['N'][l]!=0):
    
    #print(m)
    euv_time=sheet2_2['DATE'][l].split(' ')[0]+' '+sheet2_2['TIME'][l]  #将日期和时间整理拼接成字符串
    euv_peakt_2=sheet1['DATE'][m].split(' ')[0]+' '+sheet1['PEAKT'][m]  #将日期和时间整理拼接成字符串
    #和x的峰值做比较
    #print(euv_start)
    timeArray5 = time.strptime(euv_time, "%Y-%m-%d %H:%M:%S")
    timeStamp5 = int(time.mktime(timeArray5))
    #timeStamp5=timeStamp5-30
    timeArray6 = time.strptime(euv_peakt_2,"%d-%m-%Y %H:%M:%S")
    timeStamp6 = int(time.mktime(timeArray6))
    #print(timeArray5)
    #print(timeArray6)
    if sheet1['SIGN'][m]==1: #在上面筛选出来的B级耀斑中研究
        #sheet['CLASS'][i]=sheet1['CLASS'][j]
        if (timeArray6==timeArray5):
            print('time:',int(sheet2_2['TIME'][l][3:5]))
            a_1=int(sheet2_2['TIME'][l][3:5]) #把对应时间的分钟数提取出来，减掉，以前面的小时数作为横轴数据起始点
            n=l-a
            for q in range(120): 
                if (sheet2_2['VALUE'][n]==-999):
                    sheet3.loc[t,q]=np.nan
                else:
                    sheet3.loc[t,q]=sheet2_2['VALUE'][n] #这里数据差30s，这里暂时补齐，最后一起还原
                n=n+1
            t=t+1
                
            l=l+1
            m=m+1
        else:
            l=l+1
        
    else:
        m=m+1

print('sheet3的长度：')        
print(len(sheet3))






#sheet2_2_x文件，存储每日的具体euv值
print('当前m_1值：',m_1)
l_1=0

q_1=0

a_1=0
while(sheet2_2_x['B_NUM_PTS'][l_1]!=0):
    
    #print(m)
    x_time=sheet2_2_x['time_tag'][l_1]  #将日期和时间整理拼接成字符串
    x_peakt_2=sheet1['DATE'][m_1].split(' ')[0]+' '+sheet1['PEAKT'][m_1]  #将日期和时间整理拼接成字符串
    #和x的峰值做比较
    #print(euv_start)
    timeArray5 = time.strptime(x_time, "%Y-%m-%d %H:%M:%S")
    timeStamp5 = int(time.mktime(timeArray5))
    #timeStamp5=timeStamp5-30
    timeArray6 = time.strptime(x_peakt_2,"%d-%m-%Y %H:%M:%S")
    timeStamp6 = int(time.mktime(timeArray6))
    #print(timeArray5)
    #print(timeArray6)
    if sheet1['SIGN'][m_1]==1: #在上面筛选出来的B级耀斑中研究
        #sheet['CLASS'][i]=sheet1['CLASS'][j]
        if (timeArray6==timeArray5):
            print('x_time:',int(sheet2_2_x['time_tag'][l_1][14:16]))
            a_1=int(sheet2_2_x['time_tag'][l_1][14:16]) #把对应时间的分钟数提取出来，减掉，以前面的小时数作为横轴数据起始点
            n_1=l_1-a_1
            for q_1 in range(120): 
                if (sheet2_2_x['B_AVG'][n_1]==-99999):
                    sheet3_x.loc[t_1,q_1]=np.nan
                else:
                    sheet3_x.loc[t_1,q_1]=sheet2_2_x['B_AVG'][n_1] #这里数据差30s，这里暂时补齐，最后一起还原
                n_1=n_1+1
            t_1=t_1+1
                
            l_1=l_1+1
            m_1=m_1+1
        else:
            l_1=l_1+1
        
    else:
        m_1=m_1+1

print(sheet3)    
sheet3_x

In [ ]:
import datetime
import time
import xlrd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


sheet2_3 = pd.read_excel('euv_list_2013.xlsx',converters={'DATE':str,'TIME':str,'VALUE':float})
print("sheet2_3导入完成")
#导入未筛选的x数据
sheet2_3_x=pd.read_excel('goes_list_2013.xlsx',converters={'time_tag':str,'A_AVG':float,'B_AVG':float})
print("sheet2_3_x导入完成")
print('当前m值：',m)

#sheet2文件，存储每日的具体euv值

l=0

q=0

a=0
while(sheet2_3['N'][l]!=0):
    
    #print(m)
    euv_time=sheet2_3['DATE'][l].split(' ')[0]+' '+sheet2_3['TIME'][l]  #将日期和时间整理拼接成字符串
    euv_peakt_2=sheet1['DATE'][m].split(' ')[0]+' '+sheet1['PEAKT'][m]  #将日期和时间整理拼接成字符串
    #和x的峰值做比较
    #print(euv_start)
    timeArray5 = time.strptime(euv_time, "%Y-%m-%d %H:%M:%S")
    timeStamp5 = int(time.mktime(timeArray5))
    #timeStamp5=timeStamp5-30
    timeArray6 = time.strptime(euv_peakt_2,"%d-%m-%Y %H:%M:%S")
    timeStamp6 = int(time.mktime(timeArray6))
    #print(timeArray5)
    #print(timeArray6)
    if sheet1['SIGN'][m]==1: #在上面筛选出来的B级耀斑中研究
        #sheet['CLASS'][i]=sheet1['CLASS'][j]
        if (timeArray6==timeArray5):
            print('time:',int(sheet2_3['TIME'][l][3:5]))
            a_1=int(sheet2_3['TIME'][l][3:5]) #把对应时间的分钟数提取出来，减掉，以前面的小时数作为横轴数据起始点
            n=l-a
            for q in range(120): 
                if (sheet2_3['VALUE'][n]==-999):
                    sheet3.loc[t,q]=np.nan
                else:
                    sheet3.loc[t,q]=sheet2_3['VALUE'][n] #这里数据差30s，这里暂时补齐，最后一起还原
                n=n+1
            t=t+1
                
            l=l+1
            m=m+1
        else:
            l=l+1
        
    else:
        m=m+1

print('sheet3的长度：')        
print(len(sheet3))






#sheet2_3_x文件，存储每日的具体euv值
print('当前m_1值：',m_1)
l_1=0

q_1=0

a_1=0
while(sheet2_3_x['B_NUM_PTS'][l_1]!=0):
    
    #print(m)
    x_time=sheet2_3_x['time_tag'][l_1]  #将日期和时间整理拼接成字符串
    x_peakt_2=sheet1['DATE'][m_1].split(' ')[0]+' '+sheet1['PEAKT'][m_1]  #将日期和时间整理拼接成字符串
    #和x的峰值做比较
    #print(euv_start)
    timeArray5 = time.strptime(x_time, "%Y-%m-%d %H:%M:%S")
    timeStamp5 = int(time.mktime(timeArray5))
    #timeStamp5=timeStamp5-30
    timeArray6 = time.strptime(x_peakt_2,"%d-%m-%Y %H:%M:%S")
    timeStamp6 = int(time.mktime(timeArray6))
    #print(timeArray5)
    #print(timeArray6)
    if sheet1['SIGN'][m_1]==1: #在上面筛选出来的B级耀斑中研究
        #sheet['CLASS'][i]=sheet1['CLASS'][j]
        if (timeArray6==timeArray5):
            print('x_time:',int(sheet2_3_x['time_tag'][l_1][14:16]))
            a_1=int(sheet2_3_x['time_tag'][l_1][14:16]) #把对应时间的分钟数提取出来，减掉，以前面的小时数作为横轴数据起始点
            n_1=l_1-a_1
            for q_1 in range(120): 
                if (sheet2_3_x['B_AVG'][n_1]==-99999):
                    sheet3_x.loc[t_1,q_1]=np.nan
                else:
                    sheet3_x.loc[t_1,q_1]=sheet2_3_x['B_AVG'][n_1] #这里数据差30s，这里暂时补齐，最后一起还原
                n_1=n_1+1
            t_1=t_1+1
                
            l_1=l_1+1
            m_1=m_1+1
        else:
            l_1=l_1+1
        
    else:
        m_1=m_1+1

print(sheet3)    
sheet3_x

In [ ]:
import datetime
import time
import xlrd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

sheet2_4 = pd.read_excel('euv_list_2014.xlsx',converters={'DATE':str,'TIME':str,'VALUE':float})
print("sheet2_4导入完成")
#导入未筛选的x数据
sheet2_4_x=pd.read_excel('goes_list_2014.xlsx',converters={'time_tag':str,'A_AVG':float,'B_AVG':float})
print("sheet2_4_x导入完成")
print('当前m值：',m)

#sheet2文件，存储每日的具体euv值

l=0

q=0

a=0
while(sheet2_4['N'][l]!=0):
    
    #print(m)
    euv_time=sheet2_4['DATE'][l].split(' ')[0]+' '+sheet2_4['TIME'][l]  #将日期和时间整理拼接成字符串
    euv_peakt_2=sheet1['DATE'][m].split(' ')[0]+' '+sheet1['PEAKT'][m]  #将日期和时间整理拼接成字符串
    #和x的峰值做比较
    #print(euv_start)
    timeArray5 = time.strptime(euv_time, "%Y-%m-%d %H:%M:%S")
    timeStamp5 = int(time.mktime(timeArray5))
    #timeStamp5=timeStamp5-30
    timeArray6 = time.strptime(euv_peakt_2,"%d-%m-%Y %H:%M:%S")
    timeStamp6 = int(time.mktime(timeArray6))
    #print(timeArray5)
    #print(timeArray6)
    if sheet1['SIGN'][m]==1: #在上面筛选出来的B级耀斑中研究
        #sheet['CLASS'][i]=sheet1['CLASS'][j]
        if (timeArray6==timeArray5):
            print('time:',int(sheet2_4['TIME'][l][3:5]))
            a_1=int(sheet2_4['TIME'][l][3:5]) #把对应时间的分钟数提取出来，减掉，以前面的小时数作为横轴数据起始点
            n=l-a
            for q in range(120): 
                if (sheet2_4['VALUE'][n]==-999):
                    sheet3.loc[t,q]=np.nan
                else:
                    sheet3.loc[t,q]=sheet2_4['VALUE'][n] #这里数据差30s，这里暂时补齐，最后一起还原
                n=n+1
            t=t+1
                
            l=l+1
            m=m+1
        else:
            l=l+1
        
    else:
        m=m+1

print('sheet3的长度：')        
print(len(sheet3))






#sheet2_2_x文件，存储每日的具体euv值
print('当前m_1值：',m_1)
l_1=0

q_1=0

a_1=0
while(sheet2_4_x['B_NUM_PTS'][l_1]!=0):
    
    #print(m)
    x_time=sheet2_4_x['time_tag'][l_1]  #将日期和时间整理拼接成字符串
    x_peakt_2=sheet1['DATE'][m_1].split(' ')[0]+' '+sheet1['PEAKT'][m_1]  #将日期和时间整理拼接成字符串
    #和x的峰值做比较
    #print(euv_start)
    timeArray5 = time.strptime(x_time, "%Y-%m-%d %H:%M:%S")
    timeStamp5 = int(time.mktime(timeArray5))
    #timeStamp5=timeStamp5-30
    timeArray6 = time.strptime(x_peakt_2,"%d-%m-%Y %H:%M:%S")
    timeStamp6 = int(time.mktime(timeArray6))
    #print(timeArray5)
    #print(timeArray6)
    if sheet1['SIGN'][m_1]==1: #在上面筛选出来的B级耀斑中研究
        #sheet['CLASS'][i]=sheet1['CLASS'][j]
        if (timeArray6==timeArray5):
            print('x_time:',int(sheet2_4_x['time_tag'][l_1][14:16]))
            a_1=int(sheet2_4_x['time_tag'][l_1][14:16]) #把对应时间的分钟数提取出来，减掉，以前面的小时数作为横轴数据起始点
            n_1=l_1-a_1
            for q_1 in range(120): 
                if (sheet2_4_x['B_AVG'][n_1]==-99999):
                    sheet3_x.loc[t_1,q_1]=np.nan
                else:
                    sheet3_x.loc[t_1,q_1]=sheet2_4_x['B_AVG'][n_1] #这里数据差30s，这里暂时补齐，最后一起还原
                n_1=n_1+1
            t_1=t_1+1
                
            l_1=l_1+1
            m_1=m_1+1
        else:
            l_1=l_1+1
        
    else:
        m_1=m_1+1

print(sheet3)    
sheet3_x

In [ ]:
import datetime
import time
import xlrd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


sheet2_5 = pd.read_excel('euv_list_2015.xlsx',converters={'DATE':str,'TIME':str,'VALUE':float})
print("sheet2_5导入完成")
#导入未筛选的x数据
sheet2_5_x=pd.read_excel('goes_list_2015.xlsx',converters={'time_tag':str,'A_AVG':float,'B_AVG':float})
print("sheet2_5_x导入完成")
print('当前m值：',m)

#sheet2文件，存储每日的具体euv值

l=0

q=0

a=0
while(sheet2_5['N'][l]!=0):
    
    #print(m)
    euv_time=sheet2_5['DATE'][l].split(' ')[0]+' '+sheet2_5['TIME'][l]  #将日期和时间整理拼接成字符串
    euv_peakt_2=sheet1['DATE'][m].split(' ')[0]+' '+sheet1['PEAKT'][m]  #将日期和时间整理拼接成字符串
    #和x的峰值做比较
    #print(euv_start)
    timeArray5 = time.strptime(euv_time, "%Y-%m-%d %H:%M:%S")
    timeStamp5 = int(time.mktime(timeArray5))
    #timeStamp5=timeStamp5-30
    timeArray6 = time.strptime(euv_peakt_2,"%d-%m-%Y %H:%M:%S")
    timeStamp6 = int(time.mktime(timeArray6))
    #print(timeArray5)
    #print(timeArray6)
    if sheet1['SIGN'][m]==1: #在上面筛选出来的B级耀斑中研究
        #sheet['CLASS'][i]=sheet1['CLASS'][j]
        if (timeArray6==timeArray5):
            print('time:',int(sheet2_5['TIME'][l][3:5]))
            a_1=int(sheet2_5['TIME'][l][3:5]) #把对应时间的分钟数提取出来，减掉，以前面的小时数作为横轴数据起始点
            n=l-a
            for q in range(120): 
                if (sheet2_5['VALUE'][n]==-999):
                    sheet3.loc[t,q]=np.nan
                else:
                    sheet3.loc[t,q]=sheet2_5['VALUE'][n] #这里数据差30s，这里暂时补齐，最后一起还原
                n=n+1
            t=t+1
                
            l=l+1
            m=m+1
        else:
            l=l+1
        
    else:
        m=m+1

print('sheet3的长度：')        
print(len(sheet3))






#sheet2_5_x文件，存储每日的具体euv值
print('当前m_1值：',m_1)
l_1=0

q_1=0

a_1=0
while(sheet2_5_x['B_NUM_PTS'][l_1]!=0):
    
    #print(m)
    x_time=sheet2_5_x['time_tag'][l_1]  #将日期和时间整理拼接成字符串
    x_peakt_2=sheet1['DATE'][m_1].split(' ')[0]+' '+sheet1['PEAKT'][m_1]  #将日期和时间整理拼接成字符串
    #和x的峰值做比较
    #print(euv_start)
    timeArray5 = time.strptime(x_time, "%Y-%m-%d %H:%M:%S")
    timeStamp5 = int(time.mktime(timeArray5))
    #timeStamp5=timeStamp5-30
    timeArray6 = time.strptime(x_peakt_2,"%d-%m-%Y %H:%M:%S")
    timeStamp6 = int(time.mktime(timeArray6))
    #print(timeArray5)
    #print(timeArray6)
    if sheet1['SIGN'][m_1]==1: #在上面筛选出来的B级耀斑中研究
        #sheet['CLASS'][i]=sheet1['CLASS'][j]
        if (timeArray6==timeArray5):
            print('x_time:',int(sheet2_5_x['time_tag'][l_1][14:16]))
            a_1=int(sheet2_5_x['time_tag'][l_1][14:16]) #把对应时间的分钟数提取出来，减掉，以前面的小时数作为横轴数据起始点
            n_1=l_1-a_1
            for q_1 in range(120): 
                if (sheet2_5_x['B_AVG'][n_1]==-99999):
                    sheet3_x.loc[t_1,q_1]=np.nan
                else:
                    sheet3_x.loc[t_1,q_1]=sheet2_5_x['B_AVG'][n_1] #这里数据差30s，这里暂时补齐，最后一起还原
                n_1=n_1+1
            t_1=t_1+1
                
            l_1=l_1+1
            m_1=m_1+1
        else:
            l_1=l_1+1
        
    else:
        m_1=m_1+1

print(sheet3)    
sheet3_x

In [ ]:
import datetime
import time
import xlrd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


sheet2_6 = pd.read_excel('euv_list_2016.xlsx',converters={'DATE':str,'TIME':str,'VALUE':float})
print("sheet2_6导入完成")
#导入未筛选的x数据
sheet2_6_x=pd.read_excel('goes_list_2016.xlsx',converters={'time_tag':str,'A_AVG':float,'B_AVG':float})
print("sheet2_6_x导入完成")
print('当前m值：',m)

#sheet2文件，存储每日的具体euv值

l=0

q=0

a=0
while(sheet2_6['N'][l]!=0):
    
    #print(m)
    euv_time=sheet2_6['DATE'][l].split(' ')[0]+' '+sheet2_6['TIME'][l]  #将日期和时间整理拼接成字符串
    euv_peakt_2=sheet1['DATE'][m].split(' ')[0]+' '+sheet1['PEAKT'][m]  #将日期和时间整理拼接成字符串
    #和x的峰值做比较
    #print(euv_start)
    timeArray5 = time.strptime(euv_time, "%Y-%m-%d %H:%M:%S")
    timeStamp5 = int(time.mktime(timeArray5))
    #timeStamp5=timeStamp5-30
    timeArray6 = time.strptime(euv_peakt_2,"%d-%m-%Y %H:%M:%S")
    timeStamp6 = int(time.mktime(timeArray6))
    #print(timeArray5)
    #print(timeArray6)
    if sheet1['SIGN'][m]==1: #在上面筛选出来的B级耀斑中研究
        #sheet['CLASS'][i]=sheet1['CLASS'][j]
        if (timeArray6==timeArray5):
            print('time:',int(sheet2_6['TIME'][l][3:5]))
            a_1=int(sheet2_6['TIME'][l][3:5]) #把对应时间的分钟数提取出来，减掉，以前面的小时数作为横轴数据起始点
            n=l-a
            for q in range(120): 
                if (sheet2_6['VALUE'][n]==-999):
                    sheet3.loc[t,q]=np.nan
                else:
                    sheet3.loc[t,q]=sheet2_6['VALUE'][n] #这里数据差30s，这里暂时补齐，最后一起还原
                n=n+1
            t=t+1
                
            l=l+1
            m=m+1
        else:
            l=l+1
        
    else:
        m=m+1

print('sheet3的长度：')        
print(len(sheet3))






#sheet2_6_x文件，存储每日的具体euv值
print('当前m_1值：',m_1)
l_1=0

q_1=0

a_1=0
while(sheet2_6_x['B_NUM_PTS'][l_1]!=0):
    
    #print(m)
    x_time=sheet2_6_x['time_tag'][l_1]  #将日期和时间整理拼接成字符串
    x_peakt_2=sheet1['DATE'][m_1].split(' ')[0]+' '+sheet1['PEAKT'][m_1]  #将日期和时间整理拼接成字符串
    #和x的峰值做比较
    #print(euv_start)
    timeArray5 = time.strptime(x_time, "%Y-%m-%d %H:%M:%S")
    timeStamp5 = int(time.mktime(timeArray5))
    #timeStamp5=timeStamp5-30
    timeArray6 = time.strptime(x_peakt_2,"%d-%m-%Y %H:%M:%S")
    timeStamp6 = int(time.mktime(timeArray6))
    #print(timeArray5)
    #print(timeArray6)
    if sheet1['SIGN'][m_1]==1: #在上面筛选出来的B级耀斑中研究
        #sheet['CLASS'][i]=sheet1['CLASS'][j]
        if (timeArray6==timeArray5):
            print('x_time:',int(sheet2_6_x['time_tag'][l_1][14:16]))
            a=int(sheet2_6_x['time_tag'][l_1][14:16]) #把对应时间的分钟数提取出来，减掉，以前面的小时数作为横轴数据起始点
            n_1=l_1-a_1
            for q_1 in range(120): 
                if (sheet2_6_x['B_AVG'][n_1]==-99999):
                    sheet3_x.loc[t_1,q_1]=np.nan
                else:
                    sheet3_x.loc[t_1,q_1]=sheet2_6_x['B_AVG'][n_1] #这里数据差30s，这里暂时补齐，最后一起还原
                n_1=n_1+1
            t_1=t_1+1
                
            l_1=l_1+1
            m_1=m_1+1
        else:
            l_1=l_1+1
        
    else:
        m_1=m_1+1

print(sheet3)    
sheet3_x